In [1]:
!pip3 install kaggle

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
!echo '{"username":"riolaface","key":"7d0c8fac857a00875fdb3296f70c9e75"}' > kaggle.json
%rm -r /root/.kaggle
%mkdir /root/.kaggle/
%mv kaggle.json /root/.kaggle 
%ls /root/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


In [22]:
#data from https://www.kaggle.com/ahmetfurkandemr/mask-datasets-v1
!kaggle datasets download ahmetfurkandemr/mask-datasets-v1

100% 753M/754M [00:16<00:00, 46.7MB/s]
100% 754M/754M [00:16<00:00, 47.3MB/s]


In [23]:
%ls

mask-datasets-v1.zip  sample_data/


In [24]:
!unzip mask-datasets-v1.zip

Archive:  mask-datasets-v1.zip
  inflating: Mask_Datasets/Train/Mask/1.PNG  
  inflating: Mask_Datasets/Train/Mask/10.PNG  
  inflating: Mask_Datasets/Train/Mask/100.PNG  
  inflating: Mask_Datasets/Train/Mask/101.PNG  
  inflating: Mask_Datasets/Train/Mask/102.PNG  
  inflating: Mask_Datasets/Train/Mask/103.PNG  
  inflating: Mask_Datasets/Train/Mask/104.PNG  
  inflating: Mask_Datasets/Train/Mask/105.PNG  
  inflating: Mask_Datasets/Train/Mask/106.PNG  
  inflating: Mask_Datasets/Train/Mask/107.PNG  
  inflating: Mask_Datasets/Train/Mask/108.PNG  
  inflating: Mask_Datasets/Train/Mask/109.PNG  
  inflating: Mask_Datasets/Train/Mask/11.PNG  
  inflating: Mask_Datasets/Train/Mask/110.PNG  
  inflating: Mask_Datasets/Train/Mask/111.PNG  
  inflating: Mask_Datasets/Train/Mask/112.PNG  
  inflating: Mask_Datasets/Train/Mask/113.PNG  
  inflating: Mask_Datasets/Train/Mask/114.PNG  
  inflating: Mask_Datasets/Train/Mask/115.PNG  
  inflating: Mask_Datasets/Train/Mask/116.PNG  
  inflating: 

In [0]:
%rm mask-datasets-v1.zip

In [0]:
%mv Mask_Datasets/Validation/Mask/* Mask_Datasets/Train/Mask/
%mv Mask_Datasets/Validation/No_mask/* Mask_Datasets/Train/No_mask/

In [0]:
DATASET_DIR = "/content/Mask_Datasets/Train/"
BATCH_SIZE = 16

In [28]:
datagen = ImageDataGenerator(
    validation_split= 0.3, 
    rescale= 1./255,
    horizontal_flip= True,
    zoom_range=0.2,
    brightness_range= [1,2]
)

#generatore per il training
train_generator=datagen.flow_from_directory( 
    DATASET_DIR, 
    target_size= (200, 200),
    batch_size= BATCH_SIZE,
    class_mode = "binary",
    subset = "training"
)

#generatore per il test
test_generator =datagen.flow_from_directory( 
    DATASET_DIR, 
    target_size= (200, 200),
    batch_size= BATCH_SIZE,
    class_mode = "binary",
    subset = "validation"
)

print(train_generator.class_indices)

Found 665 images belonging to 2 classes.
Found 285 images belonging to 2 classes.
{'Mask': 0, 'No_mask': 1}


In [0]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=4, padding="same", activation="relu", input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=4, strides=4))
model.add(Dropout(0.5))
model.add(Conv2D(filters=32, kernel_size=4, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=4, strides=4))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [0]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(train_generator, epochs=100)

Epoch 1/100
19/42 [============>.................] - ETA: 10s - loss: 0.1570 - accuracy: 0.9461

In [35]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)
print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

18/18 [==============================] - 9s 526ms/step - loss: 0.1390 - accuracy: 0.9404
Train Accuracy = 0.9504 - Train Loss = 0.1197
Test Accuracy = 0.9404 - Test Loss = 0.1390


In [0]:
model.save("mask_recognition.h5")

In [0]:
from google.colab import files
files.download('mask_recognition.h5')